# Edge Quality Analysis
Post-hoc analysis of visibility graph edge quality after a GTSFM pipeline run.

Loads `edge_quality_report.json` and `cluster_tree.pkl` from a completed run,
then visualizes bad edges, cluster structure, and the effect of pruning.

In [ ]:
from pathlib import Path

# === CONFIGURE THESE ===
OUTPUT_ROOT = Path("../results")  # --output_root used for the pipeline run
IMAGES_DIR = Path("../benchmarks/gerrard-hall/images")  # directory with source images

# Derived paths
EDGE_QUALITY_JSON = OUTPUT_ROOT / "results" / "edge_quality_report.json"
CLUSTER_TREE_PKL = OUTPUT_ROOT / "results" / "cluster_tree.pkl"

In [ ]:
import json
import pickle

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load edge quality report
with open(EDGE_QUALITY_JSON) as f:
    report = json.load(f)

# Load cluster tree
with open(CLUSTER_TREE_PKL, "rb") as f:
    cluster_tree = pickle.load(f)

# Extract image filenames (added in our export update)
image_filenames = report.get("image_filenames", [])

# Parse edge quality into dict of (i,j) -> stats
edge_quality = {}
for edge_str, stats in report["edge_quality"].items():
    i, j = map(int, edge_str.strip("()").split(","))
    edge_quality[(i, j)] = stats

bad_edges = set()
for s in report["bad_edges"]:
    i, j = map(int, s.strip("()").split(","))
    bad_edges.add((i, j))

metadata = report["metadata"]
print(f"Total edges: {metadata['total_edges']}")
print(f"Bad edges:   {metadata['bad_edge_count']}")
print(f"Edges with no tracks: {metadata['edges_with_no_tracks']}")
print(f"Image filenames available: {len(image_filenames) > 0}")

## 1. Summary Statistics

In [ ]:
errors = []
track_counts = []
for stats in edge_quality.values():
    val = stats["mean_reproj_error_px"]
    if val != "inf":
        errors.append(val)
    track_counts.append(stats["num_tracks"])

errors = np.array(errors)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of mean reprojection errors
ax = axes[0]
ax.hist(errors, bins=50, edgecolor="black", alpha=0.7)
ax.axvline(5.0, color="red", linestyle="--", label="Bad threshold (5px)")
ax.set_xlabel("Mean Reprojection Error (px)")
ax.set_ylabel("Count")
ax.set_title("Distribution of Edge Reprojection Errors")
ax.legend()

# Histogram of track counts
ax = axes[1]
ax.hist(track_counts, bins=50, edgecolor="black", alpha=0.7, color="steelblue")
ax.set_xlabel("Number of Supporting Tracks")
ax.set_ylabel("Count")
ax.set_title("Distribution of Track Counts per Edge")

plt.tight_layout()
plt.show()

print(f"Mean reproj error:  {np.mean(errors):.2f} px")
print(f"Median reproj error: {np.median(errors):.2f} px")
print(f"Max reproj error:   {np.max(errors):.2f} px")
print(f"Edges > 5px: {np.sum(errors > 5.0)} / {len(errors)}")
print(f"Edges > 3px: {np.sum(errors > 3.0)} / {len(errors)}")
print(f"Edges > 1px: {np.sum(errors > 1.0)} / {len(errors)}")

## 2. Bad Edge Image Pairs
Showing the actual image pairs for the worst edges.

In [ ]:
def get_fname(idx):
    """Get image filename from index."""
    if idx < len(image_filenames):
        return image_filenames[idx]
    return f"image_{idx:04d}.jpg"


def load_img(img_dir, fname):
    """Load image, trying common extensions if exact match fails."""
    path = img_dir / fname
    if path.exists():
        return np.array(Image.open(path))
    for ext in [".jpg", ".jpeg", ".png", ".JPG", ".PNG"]:
        candidate = img_dir / (Path(fname).stem + ext)
        if candidate.exists():
            return np.array(Image.open(candidate))
    print(f"  WARNING: Could not find {fname} in {img_dir}")
    return np.zeros((100, 100, 3), dtype=np.uint8)


# Sort bad edges by error (worst first)
bad_edge_list = sorted(
    bad_edges,
    key=lambda e: edge_quality[e]["mean_reproj_error_px"]
    if edge_quality[e]["mean_reproj_error_px"] != "inf"
    else float("inf"),
    reverse=True,
)

n_show = min(len(bad_edge_list), 10)

if n_show > 0:
    fig, axes = plt.subplots(n_show, 2, figsize=(12, 4 * n_show))
    if n_show == 1:
        axes = axes[np.newaxis, :]

    for row, (i, j) in enumerate(bad_edge_list[:n_show]):
        stats = edge_quality[(i, j)]
        fname_i, fname_j = get_fname(i), get_fname(j)

        axes[row, 0].imshow(load_img(IMAGES_DIR, fname_i))
        axes[row, 0].set_title(f"[{i}] {fname_i}", fontsize=10)
        axes[row, 0].axis("off")

        axes[row, 1].imshow(load_img(IMAGES_DIR, fname_j))
        axes[row, 1].set_title(f"[{j}] {fname_j}", fontsize=10)
        axes[row, 1].axis("off")

        err = stats["mean_reproj_error_px"]
        err_str = f"{err:.1f}" if err != "inf" else "inf"
        axes[row, 0].set_ylabel(
            f"({i},{j}) err={err_str}px tracks={stats['num_tracks']}",
            fontsize=10, color="red", rotation=0, labelpad=120, va="center",
        )

    plt.tight_layout()
    plt.show()
else:
    print("No bad edges found!")

## 3. Graph Visualization
Interactive Plotly plots showing the visibility graph, clusters, and bad edges.

In [ ]:
# Graph visualization utilities (from partitioning.ipynb)
import plotly.graph_objects as go
from collections import deque
from typing import Any, Dict, List, Set, Tuple


def get_edge_coordinates(xy, edges):
    """Prepare edge coordinates for a Plotly Scatter trace."""
    if edges.size == 0:
        return np.array([]), np.array([])
    xe = np.empty(3 * len(edges))
    ye = np.empty(3 * len(edges))
    xe[0::3] = xy[edges[:, 0], 0]
    ye[0::3] = xy[edges[:, 0], 1]
    xe[1::3] = xy[edges[:, 1], 0]
    ye[1::3] = xy[edges[:, 1], 1]
    xe[2::3] = np.nan
    ye[2::3] = np.nan
    return xe, ye


def create_base_figure_with_background(xy, edges_arr):
    """Creates a Plotly figure with all nodes and edges as a faint background."""
    fig = go.Figure()
    xe_bg, ye_bg = get_edge_coordinates(xy, edges_arr)
    fig.add_trace(go.Scatter(
        x=xe_bg, y=ye_bg, mode="lines",
        line=dict(width=1, color="lightgray"),
        opacity=0.2, hoverinfo="none", showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=xy[:, 0], y=xy[:, 1], mode="markers",
        marker=dict(size=3, color="lightgray"),
        customdata=np.arange(len(xy)),
        hovertemplate="node %{customdata}<extra></extra>",
        showlegend=False,
    ))
    fig.update_layout(
        paper_bgcolor="white", plot_bgcolor="white",
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis=dict(visible=False),
        yaxis=dict(visible=False, scaleanchor="x", scaleratio=1),
    )
    return fig

In [ ]:
# Extract 2D camera layout for plotting
# Try COLMAP output first, then fall back to poses.pkl

xy = None

# Option A: Read from COLMAP images.txt in the reconstruction output
colmap_dirs = [
    OUTPUT_ROOT / "results" / "ba_output",
    OUTPUT_ROOT / "results" / "vggt",
    OUTPUT_ROOT / "results" / "merged",
]
for d in colmap_dirs:
    images_txt = d / "images.txt"
    if images_txt.exists():
        try:
            from gtsfm.utils.io import read_images_txt
            poses, _ = read_images_txt(str(images_txt))
            xy = np.array([p.translation()[:2] for p in poses])
            print(f"Loaded {len(poses)} poses from {images_txt}")
            break
        except Exception as e:
            print(f"Failed to read {images_txt}: {e}")

# Option B: Saved poses pickle
if xy is None:
    poses_pkl = OUTPUT_ROOT / "results" / "poses.pkl"
    if poses_pkl.exists():
        from gtsfm.utils.io import load_poses
        poses = load_poses(poses_pkl)
        xy = np.array([p.translation()[:2] for p in poses])
        print(f"Loaded {len(poses)} poses from {poses_pkl}")

if xy is None:
    print("WARNING: No camera poses found. Plotly graph visualizations will be skipped.")
    print("Expected COLMAP output in:", [str(d) for d in colmap_dirs])
else:
    N = len(xy)
    # Extract all edges from cluster tree
    all_edges = list(cluster_tree.all_edges())
    edges_arr = np.array(all_edges, dtype=int)
    valid_mask = (edges_arr[:, 0] < N) & (edges_arr[:, 1] < N)
    edges_arr = edges_arr[valid_mask]
    print(f"Total edges in cluster tree: {len(edges_arr)}, poses: {N}")

In [ ]:
# Visibility graph with bad edges highlighted in red
if xy is not None:
    fig = create_base_figure_with_background(xy, edges_arr)

    bad_edge_arr = np.array(
        [e for e in bad_edges if e[0] < N and e[1] < N], dtype=int
    )
    if len(bad_edge_arr) > 0:
        xe_bad, ye_bad = get_edge_coordinates(xy, bad_edge_arr)
        fig.add_trace(go.Scatter(
            x=xe_bad, y=ye_bad, mode="lines",
            line=dict(width=2, color="red"),
            name="Bad edges", hoverinfo="none",
        ))
        bad_nodes = np.unique(bad_edge_arr.flatten())
        fig.add_trace(go.Scatter(
            x=xy[bad_nodes, 0], y=xy[bad_nodes, 1], mode="markers",
            marker=dict(size=8, color="red", symbol="x"),
            name="Bad edge nodes",
            customdata=bad_nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(title="Visibility Graph with Bad Edges (red)")
    fig.show()
else:
    print("Skipped (no poses available)")

In [ ]:
# Original METIS leaf cluster visualization
if xy is not None:
    leaves = list(cluster_tree.leaves())
    fig = create_base_figure_with_background(xy, edges_arr)

    for idx, leaf in enumerate(leaves):
        name = f"Leaf {idx + 1}"
        nodes = np.array([k for k in leaf.all_keys() if 0 <= k < N], dtype=int)
        if nodes.size == 0:
            continue
        mask = np.isin(edges_arr[:, 0], nodes) & np.isin(edges_arr[:, 1], nodes)
        leaf_edges = edges_arr[mask]
        xe, ye = get_edge_coordinates(xy, leaf_edges)
        fig.add_trace(go.Scatter(
            x=xe, y=ye, mode="lines", line=dict(width=1),
            hoverinfo="none", name=name, legendgroup=name,
        ))
        fig.add_trace(go.Scatter(
            x=xy[nodes, 0], y=xy[nodes, 1], mode="markers",
            marker=dict(size=6), name=name, legendgroup=name,
            showlegend=False, customdata=nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(
        title="Original METIS Leaf Clusters",
        legend=dict(groupclick="togglegroup"),
    )
    fig.show()
else:
    print("Skipped (no poses available)")

## 4. Bad Edges Per Cluster

In [ ]:
leaves = list(cluster_tree.leaves())

labels = []
total_counts = []
bad_counts = []

for idx, leaf in enumerate(leaves):
    leaf_edges = set(leaf.value)
    n_bad = len(leaf_edges & bad_edges)
    labels.append(f"Leaf {idx + 1}")
    total_counts.append(len(leaf_edges))
    bad_counts.append(n_bad)

fig, ax = plt.subplots(figsize=(max(8, len(labels) * 0.8), 5))
x = np.arange(len(labels))
width = 0.35
ax.bar(x - width / 2, total_counts, width, label="Total edges", color="steelblue", alpha=0.7)
ax.bar(x + width / 2, bad_counts, width, label="Bad edges", color="red", alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel("Edge count")
ax.set_title("Bad Edges per Leaf Cluster")
ax.legend()
plt.tight_layout()
plt.show()

# Print details
for lbl, tot, bad in zip(labels, total_counts, bad_counts):
    pct = 100.0 * bad / tot if tot > 0 else 0
    print(f"  {lbl}: {bad}/{tot} bad ({pct:.0f}%)")

## 5. Re-partition After Pruning Bad Edges
Remove bad edges from the visibility graph and re-run METIS to see how the cluster tree changes.

In [ ]:
from gtsfm.graph_partitioner.metis_partitioner import MetisPartitioner
from gtsfm.products.visibility_graph import prune_edges

original_graph = sorted(cluster_tree.all_edges())
pruned_graph = prune_edges(original_graph, bad_edges)

print(f"Original: {len(original_graph)} edges")
print(f"Pruned:   {len(pruned_graph)} edges (removed {len(original_graph) - len(pruned_graph)})")
print()

partitioner = MetisPartitioner()
pruned_cluster_tree = partitioner.run(pruned_graph)

orig_leaves = list(cluster_tree.leaves())
pruned_leaves = list(pruned_cluster_tree.leaves()) if pruned_cluster_tree else []

print(f"Original: {len(orig_leaves)} leaf clusters")
print(f"Pruned:   {len(pruned_leaves)} leaf clusters")
print()
print("=== ORIGINAL ===")
print(cluster_tree)
print()
print("=== PRUNED ===")
print(pruned_cluster_tree if pruned_cluster_tree else "(empty -- graph may be disconnected)")

In [ ]:
# Pruned cluster visualization
if xy is not None and pruned_cluster_tree is not None:
    pruned_edges_arr = np.array(pruned_graph, dtype=int)
    valid_mask = (pruned_edges_arr[:, 0] < N) & (pruned_edges_arr[:, 1] < N)
    pruned_edges_arr = pruned_edges_arr[valid_mask]

    fig = create_base_figure_with_background(xy, pruned_edges_arr)

    for idx, leaf in enumerate(pruned_cluster_tree.leaves()):
        name = f"Leaf {idx + 1}"
        nodes = np.array([k for k in leaf.all_keys() if 0 <= k < N], dtype=int)
        if nodes.size == 0:
            continue
        mask = np.isin(pruned_edges_arr[:, 0], nodes) & np.isin(pruned_edges_arr[:, 1], nodes)
        leaf_edges = pruned_edges_arr[mask]
        xe, ye = get_edge_coordinates(xy, leaf_edges)
        fig.add_trace(go.Scatter(
            x=xe, y=ye, mode="lines", line=dict(width=1),
            hoverinfo="none", name=name, legendgroup=name,
        ))
        fig.add_trace(go.Scatter(
            x=xy[nodes, 0], y=xy[nodes, 1], mode="markers",
            marker=dict(size=6), name=name, legendgroup=name,
            showlegend=False, customdata=nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(
        title="Pruned Visibility Graph - Leaf Clusters",
        legend=dict(groupclick="togglegroup"),
    )
    fig.show()
else:
    print("Skipped (no poses or empty pruned tree)")